# <b>Grid Search</b>

___
## Table of Content

* Load raw features
* Train the multi-modal autoencoder
* Train the raw features models
* Validate the models
* k-nearest-neighbors class probability

In [1]:
# type: ignore

from mltoolbox.classification import DeepClassifier, KnnClassifier
from mltoolbox.representation import MultimodalAE
from sklearn.metrics import classification_report
from mltoolbox.metrics import k_class_proba_report, silhouette_report
from src.utils import get_datasets
from keras import layers
import pandas as pd
import numpy as np
import joblib

2022-12-08 17:51:42.241710: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 17:51:42.569022: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/anaconda3/envs/bigdatalab/lib/libfabric:/usr/local/cuda/lib64
2022-12-08 17:51:42.569059: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-08 17:51:42.620525: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already

In [2]:
task = 'task01' # Define the task. This variable is used mainly for savings
K = 0 # Specify the considered fold (Stratified k fold validation)

## Load raw features



In [3]:
# Load ports word2vec embeddings - entity
payload=pd.read_csv(f'../data/{task}/features/payload.csv', index_col=[0])
#payload=payload.replace({-1:0})
# Load statistics features - quantity
statistics=pd.read_csv(f'../data/{task}/features/statistics.csv', index_col=[0])
# Load statistics sequences - quantity
sequences=pd.read_csv(f'../data/{task}/features/sequences.csv', index_col=[0])
# Load ip address word2vec embeddings - entity
ipaddress=pd.read_csv(f'../data/{task}/features/ipaddress.csv', index_col=[0])
# Merge the features as raw concatenation
concat = payload.reset_index().drop(columns=['label'])\
     .merge(statistics.reset_index().drop(columns=['label']), 
            on='index', how='inner')\
     .merge(sequences.reset_index().drop(columns=['label']), 
            on='index', how='inner')\
     .merge(ipaddress.reset_index(), on='index', how='inner')\
     .set_index('index')
# Load stratified k folds
kfolds = joblib.load(f'../data/{task}/skfolds/folds.save')

In [13]:
feature, fname = concat, 'mae'
# Get the features size. Last column is the label one
P,St,Se,I = payload.shape[1]-1, statistics.shape[1]-1, \
            sequences.shape[1]-1, ipaddress.shape[1]-1

# Retrieve the training and validation samples from the k-folds order
X_train, X_val, y_train, y_val = get_datasets(kfolds, K, feature)
X_train, X_val, y_train, y_val = X_train[:100], X_val[:100], y_train[:100], y_val[:100]

In [14]:
def build_model(shapes, l1, l4):
    input_shape, P, St, Se, I = shapes
    # Define the classifier architecture
    inputs = layers.Input(input_shape,)

    # Encoder branch of modality 1 - payload embeddings
    hidden1 = layers.Lambda(lambda x: x[:, :P])(inputs)
    hidden1 = layers.Reshape((-1, 1))(hidden1)
    hidden1 = layers.Dense(64, activation='relu')(hidden1)
    hidden1 = layers.Conv1D(32, 3, activation='relu')(hidden1)
    hidden1 = layers.Dropout(.3)(hidden1)
    hidden1 = layers.MaxPooling1D(2)(hidden1)
    hidden1 = layers.Dropout(.3)(hidden1)
    hidden1 = layers.Flatten()(hidden1)
    hidden1 = layers.Dense(l1, activation='relu')(hidden1)

    # Encoder branch of modality 2 - statistics
    hidden2 = layers.Lambda(lambda x: x[:, P:St])(inputs)
    hidden2 = layers.Dense(l1, activation='relu')(hidden2)

    # Encoder branch of modality 3 - sequences
    hidden3 = layers.Lambda(lambda x: x[:, P+St:P+St+Se])(inputs)
    hidden3 = layers.Reshape((32, 4))(hidden3)
    hidden3 = layers.Conv1D(32, 3, activation='relu')(hidden3)
    hidden3 = layers.Dropout(.3)(hidden3)
    hidden3 = layers.MaxPooling1D(2)(hidden3)
    hidden3 = layers.Dropout(.3)(hidden3)
    hidden3 = layers.Flatten()(hidden3)
    hidden3 = layers.Dense(l1, activation='relu')(hidden3)

    # Encoder branch of modality 4 - ip address embeddings
    hidden4 = layers.Lambda(lambda x: x[:, P+St+Se:P+St+Se+I])(inputs)
    hidden4 = layers.Dense(l1, activation='relu')(hidden4)

    # Concatenate
    hidden = layers.Concatenate()([hidden1, hidden2, hidden3, hidden4])
    # Common encoder
    hidden = layers.Dense(512, activation='relu')(hidden)
    hidden = layers.Dense(256, activation='relu')(hidden)
    # Bottleneck
    hidden = layers.Dense(l4, activation='relu', name='Coded')(hidden)
    # Common decoder
    hidden = layers.Dense(256, activation='relu')(hidden)
    hidden = layers.Dense(512, activation='relu')(hidden)
    hidden = layers.Dense(l1*4, activation='relu')(hidden)

    hidden1 = layers.Dense(l1, activation='relu')(hidden)
    hidden1 = layers.Dense(480, activation='relu')(hidden1)
    hidden1 = layers.Reshape((15, 32))(hidden1)
    hidden1 = layers.Conv1D(32, 3, strides=2, activation="relu", padding="same")(hidden1)
    hidden1 = layers.UpSampling1D(2)(hidden1)
    hidden1 = layers.Conv1D(4, 3, activation="relu", padding="same")(hidden1)
    hidden1 = layers.UpSampling1D(2)(hidden1)
    hidden1 = layers.Flatten()(hidden1)
    output1 = layers.Dense(P, activation='linear', name='payload')(hidden1)

    hidden2 = layers.Dense(l1, activation='relu')(hidden)
    output2 = layers.Dense(St, activation='linear', name='statistics')(hidden2)

    hidden3 = layers.Dense(l1, activation='relu')(hidden)
    hidden3 = layers.Dense(480, activation='relu')(hidden3)
    hidden3 = layers.Reshape((15, 32))(hidden3)
    hidden3 = layers.Conv1D(32, 3, strides=2, activation="relu", padding="same")(hidden3)
    hidden3 = layers.UpSampling1D(2)(hidden3)
    hidden3 = layers.Conv1D(4, 3, activation="relu", padding="same")(hidden3)
    hidden3 = layers.UpSampling1D(2)(hidden3)
    hidden3 = layers.Flatten()(hidden3)
    output3 = layers.Dense(Se, activation='linear', name='sequences')(hidden3)

    hidden4 = layers.Dense(l1, activation='relu')(hidden)
    output4 = layers.Dense(I, activation='linear', name='ipaddress')(hidden4)

    outputs = [output1, output2, output3, output4]

    # Mean Squared Errors
    loss = {'payload':'mse', 'statistics':'mse', 'sequences':'mse', 'ipaddress':'mse'} 
    weights = {'payload':1/P, 'statistics':1/St, 'sequences':1/Se, 'ipaddress':1/I}

    return inputs, outputs, loss, weights

## Train the multi-modal autoencoder

In [25]:
for l1 in [32, 64, 128, 256]:
    for l4 in [32, 64, 128, 256]:
        shapes = X_train.shape[1], P, St, Se, I
        inputs, outputs, loss, weights = build_model(shapes, l1, l4)
        # Initialize the classifier
        mae = MultimodalAE(model_path=f'../data/{task}/mae/gridsearch_{l1}_{l4}_k{K}',
                        io=(inputs, outputs), losses=loss, weights=weights)
        # Fit the multi-modal autoencoder
        mae.fit(training_data=(X_train, X_train), y_sizes=[P, St, Se, I], batch_size=256, 
                scale_data=True, epochs=3, validation_data=(X_val, X_val), save=False, 
                verbose=1)

        embeddings = np.vstack([mae.transform(X_train), mae.transform(X_val)])
        # Save the embeddings
        embeddings = pd.DataFrame(embeddings, index=np.hstack(kfolds[K][:2])[:200])
        embeddings[['label']] = np.hstack(kfolds[K][2:])[:200]
        embeddings.to_csv(f'../data/{task}/embeddings/gridsearch_{l1}_{l4}_k{K}.csv')
        embeddings=pd.read_csv(f'../data/{task}/embeddings/gridsearch_{l1}_{l4}_k{K}.csv', 
                                index_col=[0])

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


1/1 [==============================] - ETA: 0s - loss: 0.0643 - payload_loss: 0.9997 - statistics_loss: 0.7506 - sequences_loss: 0.8980 - ipaddress_loss: 1.0004WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f785b797d40> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 4s 4s/step - loss: 0.0645 - payload_loss: 1.0022 - statistics_loss: 0.7535 - sequences_loss: 0.8988 - ipaddress_loss: 1.0025 - val_loss: 0.0869 - val_payload_loss: 0.9936 - val_statistics_loss: 1.6697 - val_sequences_loss: 2.6650 - val_ipaddress_loss: 0.7599
Epoch 2/3
1/1 [==============================] - 0s 97ms/step - loss: 0.0636 - payload_loss: 0.9996 - statistics_loss: 0.7387 - sequences_loss: 0.8986 - ipaddress_loss: 0.9635 - val_loss: 0.0847 - val_payload_loss: 0.9887 - val_statistics_loss: 1.6132 - val_sequences_loss: 2.6636 - val_ipaddress_loss: 0.6808
Epoch 3/3
1/1 [==============================] - 0s 93ms/step - loss: 0.0612 - pa

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - ETA: 0s - loss: 0.0643 - payload_loss: 0.9989 - statistics_loss: 0.7521 - sequences_loss: 0.8991 - ipaddress_loss: 1.0004WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f786a4c0ef0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 4s 4s/step - loss: 0.0643 - payload_loss: 0.9989 - statistics_loss: 0.7521 - sequences_loss: 0.8991 - ipaddress_loss: 1.0004 - val_loss: 0.0864 - val_payload_loss: 0.9900 - val_statistics_loss: 1.6642 - val_sequences_loss: 2.6629 - val_ipaddress_loss: 0.7417
Epoch 2/3
1/1 [==============================] - 0s 90ms/step - loss: 0.0632 - payload_loss: 0.9962 - statistics_loss: 0.7278 - sequences_loss: 0.8987 - ipaddress_loss: 0.9542 - val_loss: 0.0837 - val_payload_loss: 0.9813 - val_statistics_loss: 1.6011 - val_sequences_loss: 2.6628 - val_ipaddress_loss: 0.6372
Epoch 3/3
1/1 [==============================] - 0s 86ms/step - loss: 0.0602 - pa